In [9]:
import pandas as pd
import cbbpy
import cbbpy.womens_scraper as s
import requests
from bs4 import BeautifulSoup
import re
import requests
from datetime import datetime, timedelta




In [10]:
#date range for March Madness 2024
start_date = datetime(2024, 3, 20)
end_date = datetime(2024, 4, 7)

#define empty list to collect game ids
game_ids = set()

#loop through march mardness, find all women's basketball games that occurred on the ESPN website
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?dates={date_str}"
    
    response = requests.get(url)
    data = response.json()

    #get the game id from the page, and append to list
    events = data.get("events", [])
    for event in events:
        game_id = event.get("id")
        if game_id:
            game_ids.add(game_id)

    current_date += timedelta(days=1)


print(f"Total game IDs found: {len(game_ids)}")
print(game_ids)




Total game IDs found: 67
{'401637562', '401637553', '401637541', '401637585', '401637554', '401637578', '401637539', '401637551', '401637566', '401637588', '401637590', '401637605', '401637613', '401637607', '401637555', '401637586', '401637563', '401637557', '401637542', '401637561', '401637543', '401637556', '401637600', '401637550', '401637560', '401637552', '401637611', '401637545', '401637610', '401637564', '401637565', '401637580', '401637602', '401637583', '401637604', '401637575', '401637558', '401637606', '401637572', '401637540', '401637547', '401637544', '401637567', '401637577', '401637581', '401637576', '401637571', '401637574', '401637582', '401637589', '401637546', '401637569', '401637601', '401637587', '401637579', '401637603', '401637559', '401637612', '401637549', '401637570', '401637548', '401637584', '401637599', '401637609', '401637573', '401637608', '401637568'}


In [11]:
def extract_data(game_id:str, files:list):
	'''
	Function to extract play-by-play data for a women's basketball game (given the game_ids)

	Parameters
	----------
	game_id: str
		game id given by ESPN, taken from URL of game details page on ESPN
	files: list
		pre-defined list, intended to collect the name of each game (aka the file name used to identify it)

	Returns
	-------
	None
	'''
	pbp_df = s.get_game_pbp(game_id)
	home = pbp_df['home_team'].unique()[0].replace(" ","")
	away = pbp_df['away_team'].unique()[0].replace(" ","")
	fname = f'game_data/{home}_{away}.csv'
	files.append(fname)
	pbp_df.to_csv(fname, index=False)


In [12]:
#extract data and store to csv, collect file name in list
fnames = []
for game in game_ids:
	extract_data(game,fnames)

In [13]:
#clean the file names, append to list to be used for later analysis
names = []
for f in fnames:
	new = f.replace('game_data/','').replace('.csv','')
	names.append(new)

In [14]:
#loop through each csv file, read into a database
df_list = []
for file in fnames:
    df = pd.read_csv(file)
    #add databases to a list
    df_list.append(df)

#combine all databases into one large database
all_games = pd.concat(df_list, ignore_index=True)
#create combined csv
all_games.to_csv("combined_data.csv", index=False)
all_games.head()

,game_id,home_team,away_team,play_desc,home_score,away_score,quarter,secs_left_qt,secs_left_reg,play_team,play_type,shooting_play,scoring_play,is_three,shooter,is_assisted,assist_player,shot_x,shot_y
0,401637562,Utah Utes,South Dakota State Jackrabbits,Jump Ball won by South Dakota State,0,0,1,600,2400,South Dakota State Jackrabbits,jump ball,False,False,False,NaN,False,NaN,NaN,NaN
1,401637562,Utah Utes,South Dakota State Jackrabbits,Foul on Brooklyn Meyer.,0,0,1,586,2386,South Dakota State Jackrabbits,foul,False,False,False,NaN,False,NaN,NaN,NaN
2,401637562,Utah Utes,South Dakota State Jackrabbits,Brooklyn Meyer Turnover.,0,0,1,586,2386,South Dakota State Jackrabbits,turnover,False,False,False,NaN,False,NaN,NaN,NaN
3,401637562,Utah Utes,South Dakota State Jackrabbits,Alissa Pili missed Layup.,0,0,1,573,2373,Utah Utes,layup,True,False,False,Alissa Pili,False,NaN,23.0,1.0
4,401637562,Utah Utes,South Dakota State Jackrabbits,South Dakota State Defensive Rebound.,0,0,1,573,2373,South Dakota State Jackrabbits,rebound,False,False,False,NaN,False,NaN,NaN,NaN
